# Restaurant Data with Ratings

### Data Dictionary

There are 5 data files that are grouped like this:

Restaurants

1. chefmoz accepts.csv - Restaurant Payment facilities
2. chefmozcuisine.csv - Restaurant cuisine
3. chefmozparking.csv - Restaurant parking facilities
4. geoplaces2.csv - Restaurant profile

User-Rating

5. rating_final.csv - Restaurant rating

In [ ]:
## Loading the data in bigquery and displaying them

select * from `sustained-node-382818.Restaurants.Restaurant_profile` limit 5;
select * from `sustained-node-382818.Restaurants.Cuisine` limit 5;
select * from `sustained-node-382818.Restaurants.Parking` limit 5;
select * from `sustained-node-382818.Restaurants.Payments` limit 5;
select * from `sustained-node-382818.Restaurants.rating_final` limit 5;

#### Data Checks

In [ ]:
## Checking if all the rows are unique

SELECT COUNT(*), COUNT (DISTINCT a.placeID)
FROM `sustained-node-382818.Restaurants.Restaurant_profile` a;

- 130, 130
- Yes all the rows are unique

In [ ]:
## Finding if all the restaurants have ratings
select count (distinct a.placeID), 
from `sustained-node-382818.Restaurants.Restaurant_profile` a
where a.placeID not in 
(
  select distinct b.placeID
  from `sustained-node-382818.Restaurants.rating_final` b 
  );

All restaurants have ratings

#### Manipulating Tables

In [ ]:
## Payments Table
CREATE VIEW `sustained-node-382818.Restaurants.Payment_method` AS
SELECT
  placeID,
  MAX(CASE WHEN rn = 1 THEN Rpayment END) AS Pay_Type1,
  MAX(CASE WHEN rn = 2 THEN Rpayment END) AS Pay_Type2,
  MAX(CASE WHEN rn = 3 THEN Rpayment END) AS Pay_Type3,
  MAX(CASE WHEN rn = 4 THEN Rpayment END) AS Pay_Type4,
  MAX(CASE WHEN rn = 5 THEN Rpayment END) AS Pay_Type5,
  MAX(CASE WHEN rn = 6 THEN Rpayment END) AS Pay_Type6,
  MAX(CASE WHEN rn = 7 THEN Rpayment END) AS Pay_Type7,
  MAX(CASE WHEN rn = 8 THEN Rpayment END) AS Pay_Type8,
  Max(rn) as Payment_method
FROM (
  SELECT
    placeID,
    Rpayment,
    ROW_NUMBER() OVER (PARTITION BY placeID ORDER BY Rpayment) AS rn
  FROM `sustained-node-382818.Restaurants.Payments`
) t
GROUP BY placeID;

##QC
select * from `sustained-node-382818.Restaurants.Payment_method` limit 5;

## Cuisine Table
CREATE VIEW `sustained-node-382818.Restaurants.Cuisine_types` AS
SELECT
  placeID,
  MAX(CASE WHEN rn = 1 THEN Rcuisine END) AS Rcuisine1,
  MAX(CASE WHEN rn = 2 THEN Rcuisine END) AS Rcuisine2,
  MAX(CASE WHEN rn = 3 THEN Rcuisine END) AS Rcuisine3,
  MAX(CASE WHEN rn = 4 THEN Rcuisine END) AS Rcuisine4,
  MAX(CASE WHEN rn = 5 THEN Rcuisine END) AS Rcuisine5,
  MAX(CASE WHEN rn = 6 THEN Rcuisine END) AS Rcuisine6,
  MAX(CASE WHEN rn = 7 THEN Rcuisine END) AS Rcuisine7,
  MAX(CASE WHEN rn = 8 THEN Rcuisine END) AS Rcuisine8,
  MAX(CASE WHEN rn = 9 THEN Rcuisine END) AS Rcuisine9,
  Max(rn) as Number_of_cuisine
FROM (
  SELECT
    placeID,
    Rcuisine,
    ROW_NUMBER() OVER (PARTITION BY placeID ORDER BY Rcuisine) AS rn
  FROM `sustained-node-382818.Restaurants.Cuisine`
) t
GROUP BY placeID;

## QC
SELECT * FROM `sustained-node-382818.Restaurants.Cuisine_types` LIMIT 5;

#### Joining all Tables

In [ ]:
CREATE OR REPLACE VIEW `sustained-node-382818.Restaurants.Restaurant_rating` AS
WITH RESTAURANTS AS
(
select a.*,
b.Rcuisine1, b.Rcuisine2, b.Rcuisine3, b.number_of_cuisine,
d.Payment_method,
c.parking_lot1 , c.parking_lot2
FROM `sustained-node-382818.Restaurants.Restaurant_profile` a
left join `sustained-node-382818.Restaurants.Cuisine_types` b ON a.placeID = b.placeID
left join `sustained-node-382818.Restaurants.Parking` c ON a.placeID = c.placeID
left join `sustained-node-382818.Restaurants.Payment_method` d ON a.placeID = d.placeID
order by 1,2
),

RATING AS
(
SELECT DISTINCT placeID, 
ROUND(AVG(rating) OVER (PARTITION BY placeID ORDER BY placeID), 2) AS rating,
ROUND(AVG(food_rating) OVER (PARTITION BY placeID ORDER BY placeID), 2) AS food_rating,
ROUND(AVG(service_rating) OVER (PARTITION BY placeID ORDER BY placeID), 2) AS service_rating
FROM `sustained-node-382818.Restaurants.rating_final`
)

SELECT A.*, B.RATING, B.FOOD_RATING, B.SERVICE_RATING
FROM RESTAURANTS A
JOIN RATING B
ON A.PLACEID = B.PLACEID ;


## Viewing the Combined Table
SELECT * FROM `sustained-node-382818.Restaurants.Restaurant_rating` LIMIT 5;

### Data Analysis

In [ ]:
SELECT count(distinct placeID) FROM `sustained-node-382818.Restaurants.Restaurant_rating` ;

130 unique restaurants are being analysed

#### Restaurant Features

##### Cuisine

In [ ]:
## Type of Cuisine

SELECT A.number_of_cuisine, COUNT(*)
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
GROUP BY 1

- There are wide variety of cuisines from fastfoods, cakeshops to regional cuisines
- 26% of restaurants served Mexican cuisine, followed by 13% bars 

In [ ]:
## Number of Cuisine

SELECT A.number_of_cuisine, COUNT(*)
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
GROUP BY 1

- 80% restaurants serve only one type of cuisine

##### Alcohol

In [ ]:
select A.alcohol, count(*)
from `sustained-node-382818.Restaurants.Restaurant_rating` A
group by 1;

- 70% of the restaurants do not serve alcohol
- 26% of restaurants serve Wine and Beer
- 4% of restaurants serve full bar

##### Price

In [ ]:
SELECT A.PRICE, COUNT(*)
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
GROUP BY 1

- 35% restaurants belong to low price category
- 46% restaurants belong to medium price category
- 19% restaurants belong to high price category

##### Ambience

In [ ]:
SELECT A.Rambience, COUNT(*)
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
GROUP BY 1

- 93% restaurants belong to Familiar category
- 7% restaurants belong to Quiet category

### Data Analysis | Ratings

In [ ]:
## Finding top 10 restaurants with the highest rating

SELECT A.placeID, A.name, A.rating, A.food_rating, A.service_rating
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
ORDER BY A.rating DESC, A.food_rating DESC, A.service_rating DESC
LIMIT 10;

Restaurant Las Mananitas has highest overall rating including food and service followed by Michiko Restaurant Japones, emilianos

In [ ]:
## Restaurants with Highest rating

SELECT A.placeID, A.name, A.rating
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
ORDER BY A.rating DESC
LIMIT 5;

Restaurant Las Mananitas has highest overall rating including food and service followed by Michiko Restaurant Japones, emilianos

In [ ]:
## Restaurants with Highest food rating

SELECT A.placeID, A.name, A.food_rating,
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
ORDER BY  A.food_rating DESC
LIMIT 5;

Interms of Food, little pizza Emilio Portes Gil, La Estrella de Dimas, Giovannis have highest rating apart from the restaurants in rating

In [ ]:
## Restaurants with Highest service rating

SELECT A.placeID, A.name, A.service_rating,
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A
ORDER BY  A.service_rating DESC
LIMIT 5;

- Interms of service, cafe punta del cielo, El cotorreo have highest rating apart from the restaurants in rating

- These findings showcase that, though Restaurant Las Mananitas and Michiko Restaurant Japones top performs in rating but when in comes to food and service there are other top performers

In [ ]:
## Bucketing Restaurants based on thier ratings 

CREATE OR REPLACE VIEW `sustained-node-382818.Restaurants.Restaurant_rating_bucket` AS
SELECT A.*, 
  CASE
    WHEN A.rating <= 1 THEN 'BAD'
    WHEN A.rating > 1 AND A.rating <= 1.5 THEN 'GOOD'
    WHEN A.rating > 1.5 AND A.rating <= 2 THEN 'EXCELLENT'
    ELSE 'NULL'
  END AS bucket,
  CASE
    WHEN A.Food_rating <= 1 THEN 'BAD'
    WHEN A.Food_rating > 1 AND A.Food_rating <= 1.5 THEN 'GOOD'
    WHEN A.Food_rating > 1.5 AND A.Food_rating <= 2 THEN 'EXCELLENT'
    ELSE 'NULL'
  END AS Food_bucket,
  CASE
    WHEN A.service_rating <= 1 THEN 'BAD'
    WHEN A.service_rating > 1 AND A.service_rating <= 1.5 THEN 'GOOD'
    WHEN A.service_rating > 1.5 AND A.service_rating <= 2 THEN 'EXCELLENT'
    ELSE 'NULL'
  END AS service_bucket,
FROM `sustained-node-382818.Restaurants.Restaurant_rating` A ;

In [ ]:
SELECT A. BUCKET, COUNT(*)
FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket` A
GROUP BY 1;

- 14% of restuarants had ratings greater than 1.5
- 48% of restuarants had ratings between 1 and 1.5
- 38% of restuarants had ratings less than 1

### Data Analysis | Ratings - Deep Dive

In [ ]:
## Checking relationship among price, ambience, smoking area, cuisine and ratings

WITH counts AS (
  SELECT a.dress_code,
  -- a.price, a.Rambience, a.area, a.smoking_area, a.number_of_cuisine, a.Rcuisine1, a.dress_code
  COUNT(*) AS count
  FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket` a
  WHERE bucket IN ('EXCELLENT','GOOD')
  GROUP BY 1
)
SELECT c.dress_code,
-- c.price, c.Rambience, c.area, c.smoking_area, c.number_of_cuisine, c.Rcuisine1, c.dress_code
c.count, ROUND(c.count / total.total_count * 100, 0) AS percent
FROM counts c
JOIN (SELECT COUNT(*) AS total_count
      FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket`
      WHERE bucket IN ('EXCELLENT','GOOD')) total
ON 1=1
ORDER BY percent desc;

- 52% of restaurants in Excellent and Good category were mid priced restaurants
- 95% of restaurants in Excellent and Good category had 'Familiar' ambience
- 89% the restaurants were of Closed area, with excellent rated restaurants being only closed
- 54% of restaurants in Excellent and Good category category had no smoking area
- 59% of restaurants served only one cuisine
- For 27% of the restaurants, cusine is not known. 15% had Mexican cuisine followed by Bar 

### Data Analysis | Food Ratings - Deep Dive

In [ ]:
SELECT A. FOOD_BUCKET, COUNT(*)
FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket` A
GROUP BY 1;

- 12% of restuarants had ratings greater than 1.5
- 59% of restuarants had ratings between 1 and 1.5
- 29% of restuarants had ratings less than 1

In [ ]:
## Checking relation among smoking area, alcohol , cuisine and Food ratings

WITH counts AS (
  SELECT a.Rcuisine1,
  -- a.alcohol, a.smoking_area, a.number_of_cuisine, a.Rcuisine1, 
  COUNT(*) AS count
  FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket` a
  WHERE food_bucket IN ('EXCELLENT','GOOD')
  GROUP BY 1
)
SELECT c.Rcuisine1,
--  c.alcohol ,c.smoking_area, c.number_of_cuisine, c.Rcuisine1, 
c.count, ROUND(c.count / total.total_count * 100, 0) AS percent
FROM counts c
JOIN (SELECT COUNT(*) AS total_count
      FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket`
      WHERE food_bucket IN ('EXCELLENT','GOOD')) total
ON 1=1;

- 62% of restaurants in Excellent and Good category were 'No Alcohol served'
- 52% of restaurants in Excellent and Good category didnt not have smoking area
- 64% the restaurants in Excellent and Good category served only one cuisine
- For 25% of the restaurants, cusine is not known. 22% had Mexican cuisine followed by Bar 

### Data Analysis | Service Ratings - Deep Dive

In [ ]:
SELECT A. SERVICE_BUCKET, COUNT(*)
FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket` A
GROUP BY 1;

- 10% of restuarants had ratings greater than 1.5
- 37% of restuarants had ratings between 1 and 1.5
- 53% of restuarants had ratings less than 1

In [ ]:
## Checking relation among parking, payment , other services and service ratings

WITH counts AS (
  SELECT a.parking_lot2,
  -- a.other_services, a.payment_methods, a.parking_lot1, a.parking_lot2, 
  COUNT(*) AS count
  FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket` a
  WHERE service_bucket IN ('EXCELLENT','GOOD')
  GROUP BY 1
)
SELECT c.parking_lot2,
--  c.other_services, c.payment_methods, c.parking_lot1, c.parking_lot2,
c.count, ROUND(c.count / total.total_count * 100, 0) AS percent
FROM counts c
JOIN (SELECT COUNT(*) AS total_count
      FROM `sustained-node-382818.Restaurants.Restaurant_rating_bucket`
      WHERE service_bucket IN ('EXCELLENT','GOOD')) total
ON 1=1;

- 87% of restaurants in Excellent and Good category did not have any other services
- 55% of restaurants had only one payment method whereas only 34% of restaurants in Excellent and Good category had only one payment method
- 48% of restaurants in Excellent and Good category did not have any parking facility

In [ ]:
### Identifying Correlation between ratings, cuisines and number of payment method available

SELECT
  CORR(rating, number_of_cuisine) AS corr_cuisines,
  CORR(rating, Payment_method) AS corr_payment,
  CORR(food_rating, number_of_cuisine) AS corr_cuisines_food,
  CORR(service_rating, Payment_method) AS corr_payment_service
FROM  `sustained-node-382818.Restaurants.Restaurant_rating_bucket`;

- correlation between restaurant rating and number of cuisine served = 0.10064901223819836
- correlation between restaurant rating and number of payment method available =  0.27159236397050907
- correlation between restaurant food rating and number of cuisine served = -0.07932895719755062
- correlation between restaurant service rating and number of payment method available =  0.34756923224914443

- This suggests that, the number of cuisines served does not strongly influence the restaurant rating.
- There is a tendency for higher-rated restaurants to offer a greater variety of payment methods

### Key Questions

1. Is there any correlation between the ratings and other attributes ?
- The number of cuisines served does not strongly influence the restaurant rating.
- There is a tendency for higher-rated restaurants to offer a greater variety of payment methods

2. How does the rating vary across different cuisines? Are there any cuisines that generally receive higher ratings?
- Mexican followed by bar generally receive higher ratings while regional and japanese had low ratings

3. Are there any noticeable differences in ratings between restaurants with different characteristics (e.g., informal vs. formal, familiar vs. closed)?
- Yes. 97% of the restaurants that had high ratings had informal or causual dress code and 
- 95% with 'Familiar ambience' and 89% with 'closed' area

4. How does price of the restaurants affect the rating?
- More than 50% of restaurants that received higher ratings are medium priced restaurants

### Insights / Recommendations

1. Pricing : Mid priced restaurants attracts higher ratings from customers. Customer more likely to pay medium price for their dining experinece
2. Ambience and area : 95% and 89% of Familiar ambience and closed space restaurants received higher ratings, this suggest that customer prefer comfortable, welcoming and closed type atmosphere 
3.  A considerable percentage 54% of restaurants in the 'Excellent' and 'Good' categories had no smoking area. This reflects the growing trend of providing smoke-free dining spaces to cater to customer preferences and health concerns.
4. Cuisine: A significant proportion 59% of restaurants with higher ratings served only one cuisine. This highlights the importance of specialization and focusing on delivering high-quality dishes within a specific culinary genre.
5. Unknown Cuisine: It is worth noting that for 27% of the restaurants, the cuisine information is not known. This suggests a need for better data collection and categorization in order to gain a comprehensive understanding of the restaurant landscape.
6. 53% of restaurants had lower service ratings, this implies that restaurants should had more services to enhance customer experience